In [1]:
#p_info.csv  -> dob,height,playerid
#m_info.csv -> mkey,date,home/away#_id/pos,home/away_formation/manager/team,result,score
#t_info.csv -> team_id,name,seats
#p_values.csv -> date,playerid,y
#m_odds.csv -> at,date,ht,odds,odds1,odds2,oddsx,score
import pandas as pd
import numpy as np
from datetime import datetime
import math
import time

In [2]:
datafolder = '../data_sources/soccerway_data/'

### available at ETL:

In [3]:
p_i_df = pd.read_csv(datafolder + 'p_info.csv').drop_duplicates()
t_i_df = pd.read_csv(datafolder + 't_info.csv').drop_duplicates()
p_v_df = pd.read_csv(datafolder + 'p_values.csv').drop_duplicates()
m_o_df = pd.read_csv(datafolder + 'm_odds.csv').drop_duplicates()
m_i_df = pd.read_csv(datafolder + 'm_info.csv').drop_duplicates()

In [4]:
p_i_df

,dob,height,playerid
0,1980-09-23,170,1
1,1968-12-18,186,2
2,1967-12-29,180,3
3,1968-03-09,179,4
4,1973-05-17,191,5
5,1973-08-06,181,6
6,1978-07-08,179,7
7,1973-07-29,192,8
8,1995-03-29,195,262153
9,1978-06-09,184,10


In [5]:
#teszt=t_i_df.merge(m_i_df[b], left_on='team_id', right_on='home_team')
#teszt.merge(p_i_df[['height','playerid']], left_on='home1_id', right_on='playerid')
t_i_df

,team_id,name,seats
0,1,1.FC Saarbrücken,30931.0
1,2,1.FC Kaiserslautern,49780.0
2,3,1. FC Köln,50000.0
3,4,1.FC Nuremberg,50000.0
4,5,AC Milan,80018.0
5,6,Adanaspor,16095.0
6,7,El Ahly Cairo,16000.0
7,8,Alemannia Aachen,32960.0
8,9,PS Kalamata,4496.0
9,10,Arminia Bielefeld,26515.0


In [6]:
p_v_df.drop_duplicates()

,date,playerid,y
0,2004-10-04,1,250000
1,2007-06-19,1,200000
2,2009-04-23,1,0
3,2004-10-04,2,0
4,2005-07-27,2,250000
5,2004-10-04,3,0
6,2004-10-04,4,400000
7,2005-02-19,4,300000
8,2006-09-27,4,400000
9,2006-10-13,4,300000


In [7]:
p_v_df

,date,playerid,y
0,2004-10-04,1,250000
1,2007-06-19,1,200000
2,2009-04-23,1,0
3,2004-10-04,2,0
4,2005-07-27,2,250000
5,2004-10-04,3,0
6,2004-10-04,4,400000
7,2005-02-19,4,300000
8,2006-09-27,4,400000
9,2006-10-13,4,300000


In [8]:
m_o_df[['at']].unstack().value_counts()

Al                        792
Atletico                  354
Osaka                     309
Everton                   272
Santos                    267
River Plate               263
Albirex Niigata           247
Brest                     243
Benfica                   232
U. Catolica               226
Paris SG                  222
Guarani                   222
PR                        219
MG                        217
FC Porto                  211
Lyon                      209
Basel                     209
Marseille                 207
Sao Paulo                 202
FC Steaua Bucuresti       201
Rudar                     200
Botafogo RJ               197
Fenerbahce                196
Palmeiras                 192
Inter                     192
Chelsea                   191
Trabzonspor               189
U.N.A.M.                  189
Zurich                    187
Derby                     187
                         ... 
USA U16                     1
Real Hyria                  1
Torns (Swe

In [9]:
m_i_df

,mkey,away10_id,away10_pos,away11_id,away11_pos,away1_id,away1_pos,away2_id,away2_pos,away3_id,...,home7_pos,home8_id,home8_pos,home9_id,home9_pos,home_formation,home_manager,home_team,result,score
0,2604022,45146,top: 2%; left: 30%;,66521,top: 22%; left: 40%;,34370,top: 80%; left: 40%;,262523,top: 61%; left: 73%;,88682,...,top: 33%; left: 55%;,240306,top: 36%; left: 40%;,40428,top: 33%; left: 25%;,3-5-2 flat,2127,410,2,2-3
1,948786,113974,top: 80%; left: 40%;,113979,top: 61%; left: 7.5%;,59373,top: 61%; left: 73%;,57628,top: 63%; left: 28%;,68152,...,top: 18%; left: 65%;,39743,top: 3%; left: 40%;,38267,top: 63%; left: 28%;,4-2-3-1,97,3817,0,1-1
2,1134605,88983,top: 2%; left: 30%;,143219,top: 2%; left: 50%;,45601,top: 80%; left: 40%;,145922,top: 61%; left: 73%;,128912,...,top: 43%; left: 52%;,128262,top: 63%; left: 52.5%;,42413,top: 43%; left: 28%;,4-2-3-1,3866,9251,1,4-0
3,2463048,58030,top: 18%; left: 65%;,78147,top: 3%; left: 40%;,93763,top: 80%; left: 40%;,56112,top: 43%; left: 40%;,45677,...,top: 43%; left: 28%;,85927,top: 43%; left: 52%;,39998,top: 23%; left: 68%;,4-4-2 double 6,433,269,1,1-0
4,2547389,183120,top: 23%; left: 40%;,42393,top: 3%; left: 40%;,128969,top: 80%; left: 40%;,110668,top: 61%; left: 7.5%;,19312,...,top: 18%; left: 15%;,22378,top: 61%; left: 7.5%;,24049,top: 43%; left: 52%;,4-2-3-1,5191,339,1,1-0
5,2463041,41118,top: 63%; left: 52.5%;,18296,top: 3%; left: 40%;,89649,top: 80%; left: 40%;,31608,top: 63%; left: 28%;,83415,...,top: 18%; left: 65%;,89654,top: 23%; left: 40%;,84425,top: 18%; left: 15%;,4-2-3-1,6092,17,0,1-1
6,1143249,6053,top: 2%; left: 50%;,33570,top: 2%; left: 30%;,27539,top: 80%; left: 40%;,58816,top: 61%; left: 73%;,58821,...,top: 35%; left: 55%;,53556,top: 25%; left: 12%;,7153,top: 35%; left: 25%;,4-5-1,789,2381,2,0-4
7,1143248,16499,top: 18%; left: 65%;,174984,top: 3%; left: 40%;,14555,top: 80%; left: 40%;,44996,top: 63%; left: 28%;,16353,...,top: 43%; left: 28%;,44992,top: 43%; left: 52%;,5628,top: 10%; left: 15%;,4-3-3 def.,806,3216,2,1-3
8,1143245,37091,top: 2%; left: 30%;,38768,top: 2%; left: 50%;,28124,top: 80%; left: 40%;,4873,top: 63%; left: 28%;,7402,...,top: 61%; left: 7.5%;,21494,top: 28%; left: 53%;,56372,top: 10%; left: 65%;,4-3-3 off.,2043,820,1,2-1
9,1143247,36873,top: 18%; left: 15%;,33833,top: 3%; left: 40%;,7110,top: 80%; left: 40%;,16698,top: 61%; left: 73%;,27280,...,top: 23%; left: 68%;,80696,top: 43%; left: 52%;,8539,top: 43%; left: 28%;,4-4-2 double 6,5199,2323,1,1-0


In [5]:
a=[]
b=[]
for k in range(1,12):
    a.append('home%s_id' %k)
    a.append('home%s_pos' %k)
    a.append('away%s_id' %k)
    a.append('away%s_pos' %k)
    b.append('home%s_id' %k)

def stringtoint(a):
    if type(a)==str:
        a=a.replace(';','')
        a=a.replace(' ','')
        a=a.replace(',','')
        a=a.replace('.','')
    try:
        a=int(a)
    except:
        a=float('nan')
    
    return a

In [11]:
m_i_df[b]#.sort_values('home_team')

,home1_id,home2_id,home3_id,home4_id,home5_id,home6_id,home7_id,home8_id,home9_id,home10_id,home11_id
0,16408,261611,6750,169052,125251,129893,108484,240306,40428,6031,29316
1,3612,19354,26485,35208,38434,31469,35872,39743,38267,16474,31173
2,73472,141655,118534,110798,141650,118535,122153,128262,42413,62049,61887
3,18971,57966,11583,57101,2169,3993,48633,85927,39998,31968,48547
4,91931,57987,101304,58000,248934,25758,58949,22378,24049,97567,59908
5,104220,45685,134636,153664,77812,138091,87650,89654,84425,5036,164265
6,67797,85302,29678,21086,33569,34531,90970,53556,7153,10364,49838
7,16693,138409,33794,35029,7072,39711,74648,44992,5628,15821,66405
8,85339,37722,7005,10504,39564,44608,85783,21494,56372,31867,51350
9,22109,7152,138476,14514,47310,30846,5516,80696,8539,46658,6924


In [12]:
m_i_df[['home_formation','away_formation']].unstack().value_counts().index[0]

'4-2-3-1'

In [13]:
m_i_df[['away10_id','away11_id','away1_id','away2_id','away3_id','away4_id','away5_id','away6_id','away7_id','away8_id','away9_id','home10_id','home11_id','home1_id','home2_id','home3_id','home4_id','home5_id','home6_id','home7_id','home8_id','home9_id']].unstack().value_counts()

2219      609
2514      516
17259     514
10        510
27513     501
6710      496
8198      464
3979      462
61885     457
77        451
23951     442
25557     436
27520     433
27796     431
17965     426
26        415
89        415
3332      413
28003     411
79531     409
1662      401
27765     399
5285      397
58358     395
2963      390
27521     388
35664     388
77388     387
6020      387
27799     382
         ... 
453850      1
44060       1
123957      1
287878      1
105529      1
208780      1
16703       1
239938      1
281731      1
279682      1
211280      1
285825      1
196951      1
221531      1
205934      1
192873      1
89121       1
285826      1
87072       1
391560      1
33818       1
7177        1
336275      1
400586      1
349347      1
293304      1
312465      1
271501      1
267403      1
361223      1
Length: 141746, dtype: int64

In [14]:
#number-of-players-with-no-games
#player-with-highest-number-of-games

In [15]:
hanymeccs=m_i_df[['away10_id','away11_id','away1_id','away2_id','away3_id','away4_id','away5_id','away6_id','away7_id','away8_id','away9_id','home10_id','home11_id','home1_id','home2_id','home3_id','home4_id','home5_id','home6_id','home7_id','home8_id','home9_id']].unstack().value_counts().sort_index()

#player-with-highest-number-of-games
hanymeccs.sort_values(ascending=False).index[0]

2219

In [16]:
osszesjatekos=p_i_df.sort_values(by='playerid')
osszesjatekos


,dob,height,playerid
0,1980-09-23,170,1
1,1968-12-18,186,2
2,1967-12-29,180,3
3,1968-03-09,179,4
4,1973-05-17,191,5
5,1973-08-06,181,6
6,1978-07-08,179,7
7,1973-07-29,192,8
9,1978-06-09,184,10
10,1972-02-03,191,11


In [17]:
nemjatszott=0
for i in range(len(osszesjatekos)):
    if int(osszesjatekos.iloc[i,2]) not in hanymeccs.index:
        nemjatszott+=1

In [18]:
#number-of-players-with-no-games
nemjatszott

227

player-with-highest-number-of-games-where-his-team-didnt-concede - nem kapott gólt a csapat - kész

most-games-played-in-same-position-by-player - kész 

most-different-positions-by-player - kész

most-different-formations-by-player - kész

largest-odds-overcome-in-game - kész
legnagyobb olyan beteljesült odds ami nem döntetlen. (csak azok közül a meccsek közül ami az input adatbázisban szerepelnek)

largest-height-difference-overcome-in-game - M_I_DF-BŐL HOME&AWAY ID + MKEY + PIDF-BŐL HEIGHT - PLAYERID KÖTI ÖSSZE
csapatok átlagmagassága közötti különbség


longest-time-in-days-between-two-games-for-player - PD.DATETIME + MIDF SORBARAKVA DATE ALAPJÁN + TIMESTAMP MIN MEGKERESNI ÉS ÁTSZÁMOLNI NAPOKRA + FELTÉTEL: VALAMELYIK PLAYER ID EGYEZZEN 

biggest-value-difference - AVERAGE HEIGHT + TEAM IDK + P_V_DF + MIDF CSAPATOK ALAPJÁN MECCSENKÉNTI ÉRTÉKKÜLÖNBSÉG
csapatok összesített játékos-értéke közötti különbség

biggest-value-difference-upset # an upset means the unexpected team won - P_V_DF + MIDF CSAPATOK ALAPJÁN MECCSENKÉNTI ÉRTÉKKÜLÖNBSÉG - FELTÉTEL: A NAGYOBB ODDSAL RENDELKEZŐ CSAPAT NYER

biggest-value-difference-with-higher-odds - SKIP
legnagyobb játékos-érték beli különbség, ahol az értékesebb csapatra van nagyobb odds

biggest-stadium-capacity-difference-upset - TIDF + MIDF MATCHING TEAM ID ALAPJÁN
kisebb stadionú csapat nyert


capacity-of-stadium-of-team-with-most-games - MIDF + TIDF 

id-of-oldest-team-to-win-a-game - M_O_DF + M_I_DF + T_I_DF MODF BŐL DATETIME LEGRÉGEBBI MECCSEK, ÖSSZEMATCHELÉSE A TIDF-EL HOGY LEGYEN TEAM ID, MAJD ÖSSZEMATCHELÉSE AZ MIDF-EL HOGY LÁSSUK CSAPATONKÉNT A LEGÚJABB ÉS LEGRÉGEBBI MECCSEK KÖZTI KÜLÖNBSÉGET
a győztes meccs idején hány éves volt?
igen, nyilván a kor mindig a meccs időpontjában értelmezendő

biggest-age-difference-between-teams-match-id - SKIP

median-of-winning-team-average-age - M_I_DF + P_I_DF + PD.DATETIME

median-of-favorite-team-average-age - MIDF + PIDF + MODF + PD.DATETIME
#favorite means has lower odds of winning

median-of-underdog-team-average-age  - MIDF + PIDF + MODF + PD.DATETIME
#underdog means has higher odds of winning

team-with-most-wins-as-underdog - M_O_DF

team-with-most-losses-as-favorite - MODF

team-with-lowest-average-odds-of-draw - MODF

position-with-highest-average-value - AVERAGE HEIGHT + P_V_DF PLAYERID ALAPJÁN, IDŐPONTONKÉNT GROUPBYOLNI A POZICIÓK ALAPJÁN A JÁTÉKOSOK ÉRTÉKÉT

position-with-largest-average-height - kész

position-with-youngest-average-age - AVERAGE HEIGHT + PD.DATETIME TIMESTAMP

In [19]:
#player-with-highest-number-of-games-where-his-team-didnt-concede - nem kapott gólt a csapat
m_i_df[['score']].unstack().value_counts()

1-1      18431
1-0      16805
2-1      14206
0-1      12569
0-0      12388
2-0      12367
1-2      10817
2-2       7887
0-2       7868
3-1       7192
3-0       6993
1-3       4698
3-2       4204
0-3       3801
2-3       3388
4-0       3219
4-1       3026
1-4       1751
4-2       1720
0-4       1578
3-3       1535
5-0       1294
2-4       1232
5-1       1096
4-3        686
0-5        614
5-2        575
1-5        565
3-4        557
6-0        502
         ...  
8-5          2
10-2         2
15-14        2
0-15         2
10-1         2
1-12         2
8-4          2
4-8          2
11-1         2
8-10         1
1-11         1
12-2         1
16-15        1
9-6          1
2-11         1
7-7          1
11-3         1
10-3         1
3-10         1
13-12        1
12-11        1
0-14         1
13-14        1
15-0         1
3-9          1
18-19        1
5-8          1
3-8          1
17-0         1
7-9          1
Length: 125, dtype: int64

In [8]:
nemnulla=[]
#for i, rows in m_i_df.iterrows():
#    nemnulla.append(m_i_df.loc[i,'score'].split('-'))


In [6]:
m_i_df['score_away'] = m_i_df['score'].str.split('-').str[0]
m_i_df['score_home'] = m_i_df['score'].str.split('-').str[1]
m_i_df

,mkey,away10_id,away10_pos,away11_id,away11_pos,away1_id,away1_pos,away2_id,away2_pos,away3_id,...,home8_pos,home9_id,home9_pos,home_formation,home_manager,home_team,result,score,score_away,score_home
0,2604022,45146,top: 2%; left: 30%;,66521,top: 22%; left: 40%;,34370,top: 80%; left: 40%;,262523,top: 61%; left: 73%;,88682,...,top: 36%; left: 40%;,40428,top: 33%; left: 25%;,3-5-2 flat,2127,410,2,2-3,2,3
1,948786,113974,top: 80%; left: 40%;,113979,top: 61%; left: 7.5%;,59373,top: 61%; left: 73%;,57628,top: 63%; left: 28%;,68152,...,top: 3%; left: 40%;,38267,top: 63%; left: 28%;,4-2-3-1,97,3817,0,1-1,1,1
2,1134605,88983,top: 2%; left: 30%;,143219,top: 2%; left: 50%;,45601,top: 80%; left: 40%;,145922,top: 61%; left: 73%;,128912,...,top: 63%; left: 52.5%;,42413,top: 43%; left: 28%;,4-2-3-1,3866,9251,1,4-0,4,0
3,2463048,58030,top: 18%; left: 65%;,78147,top: 3%; left: 40%;,93763,top: 80%; left: 40%;,56112,top: 43%; left: 40%;,45677,...,top: 43%; left: 52%;,39998,top: 23%; left: 68%;,4-4-2 double 6,433,269,1,1-0,1,0
4,2547389,183120,top: 23%; left: 40%;,42393,top: 3%; left: 40%;,128969,top: 80%; left: 40%;,110668,top: 61%; left: 7.5%;,19312,...,top: 61%; left: 7.5%;,24049,top: 43%; left: 52%;,4-2-3-1,5191,339,1,1-0,1,0
5,2463041,41118,top: 63%; left: 52.5%;,18296,top: 3%; left: 40%;,89649,top: 80%; left: 40%;,31608,top: 63%; left: 28%;,83415,...,top: 23%; left: 40%;,84425,top: 18%; left: 15%;,4-2-3-1,6092,17,0,1-1,1,1
6,1143249,6053,top: 2%; left: 50%;,33570,top: 2%; left: 30%;,27539,top: 80%; left: 40%;,58816,top: 61%; left: 73%;,58821,...,top: 25%; left: 12%;,7153,top: 35%; left: 25%;,4-5-1,789,2381,2,0-4,0,4
7,1143248,16499,top: 18%; left: 65%;,174984,top: 3%; left: 40%;,14555,top: 80%; left: 40%;,44996,top: 63%; left: 28%;,16353,...,top: 43%; left: 52%;,5628,top: 10%; left: 15%;,4-3-3 def.,806,3216,2,1-3,1,3
8,1143245,37091,top: 2%; left: 30%;,38768,top: 2%; left: 50%;,28124,top: 80%; left: 40%;,4873,top: 63%; left: 28%;,7402,...,top: 28%; left: 53%;,56372,top: 10%; left: 65%;,4-3-3 off.,2043,820,1,2-1,2,1
9,1143247,36873,top: 18%; left: 15%;,33833,top: 3%; left: 40%;,7110,top: 80%; left: 40%;,16698,top: 61%; left: 73%;,27280,...,top: 43%; left: 52%;,8539,top: 43%; left: 28%;,4-4-2 double 6,5199,2323,1,1-0,1,0


In [9]:
nemnulla1=[]
nemnulla2=[]
for i in range(len(nemnulla)):
    nemnulla1.append(nemnulla[i][0])
    nemnulla2.append(nemnulla[i][1])

In [10]:
m_i_df['score0']=nemnulla1
m_i_df['score1']=nemnulla2


ValueError: Length of values does not match length of index

In [11]:
m_i_df

,mkey,away10_id,away10_pos,away11_id,away11_pos,away1_id,away1_pos,away2_id,away2_pos,away3_id,...,home8_pos,home9_id,home9_pos,home_formation,home_manager,home_team,result,score,score_away,score_home
0,2604022,45146,top: 2%; left: 30%;,66521,top: 22%; left: 40%;,34370,top: 80%; left: 40%;,262523,top: 61%; left: 73%;,88682,...,top: 36%; left: 40%;,40428,top: 33%; left: 25%;,3-5-2 flat,2127,410,2,2-3,2,3
1,948786,113974,top: 80%; left: 40%;,113979,top: 61%; left: 7.5%;,59373,top: 61%; left: 73%;,57628,top: 63%; left: 28%;,68152,...,top: 3%; left: 40%;,38267,top: 63%; left: 28%;,4-2-3-1,97,3817,0,1-1,1,1
2,1134605,88983,top: 2%; left: 30%;,143219,top: 2%; left: 50%;,45601,top: 80%; left: 40%;,145922,top: 61%; left: 73%;,128912,...,top: 63%; left: 52.5%;,42413,top: 43%; left: 28%;,4-2-3-1,3866,9251,1,4-0,4,0
3,2463048,58030,top: 18%; left: 65%;,78147,top: 3%; left: 40%;,93763,top: 80%; left: 40%;,56112,top: 43%; left: 40%;,45677,...,top: 43%; left: 52%;,39998,top: 23%; left: 68%;,4-4-2 double 6,433,269,1,1-0,1,0
4,2547389,183120,top: 23%; left: 40%;,42393,top: 3%; left: 40%;,128969,top: 80%; left: 40%;,110668,top: 61%; left: 7.5%;,19312,...,top: 61%; left: 7.5%;,24049,top: 43%; left: 52%;,4-2-3-1,5191,339,1,1-0,1,0
5,2463041,41118,top: 63%; left: 52.5%;,18296,top: 3%; left: 40%;,89649,top: 80%; left: 40%;,31608,top: 63%; left: 28%;,83415,...,top: 23%; left: 40%;,84425,top: 18%; left: 15%;,4-2-3-1,6092,17,0,1-1,1,1
6,1143249,6053,top: 2%; left: 50%;,33570,top: 2%; left: 30%;,27539,top: 80%; left: 40%;,58816,top: 61%; left: 73%;,58821,...,top: 25%; left: 12%;,7153,top: 35%; left: 25%;,4-5-1,789,2381,2,0-4,0,4
7,1143248,16499,top: 18%; left: 65%;,174984,top: 3%; left: 40%;,14555,top: 80%; left: 40%;,44996,top: 63%; left: 28%;,16353,...,top: 43%; left: 52%;,5628,top: 10%; left: 15%;,4-3-3 def.,806,3216,2,1-3,1,3
8,1143245,37091,top: 2%; left: 30%;,38768,top: 2%; left: 50%;,28124,top: 80%; left: 40%;,4873,top: 63%; left: 28%;,7402,...,top: 28%; left: 53%;,56372,top: 10%; left: 65%;,4-3-3 off.,2043,820,1,2-1,2,1
9,1143247,36873,top: 18%; left: 15%;,33833,top: 3%; left: 40%;,7110,top: 80%; left: 40%;,16698,top: 61%; left: 73%;,27280,...,top: 43%; left: 52%;,8539,top: 43%; left: 28%;,4-4-2 double 6,5199,2323,1,1-0,1,0


In [12]:
concede=m_i_df.drop_duplicates()

In [13]:
concede.drop_duplicates()

,mkey,away10_id,away10_pos,away11_id,away11_pos,away1_id,away1_pos,away2_id,away2_pos,away3_id,...,home8_pos,home9_id,home9_pos,home_formation,home_manager,home_team,result,score,score_away,score_home
0,2604022,45146,top: 2%; left: 30%;,66521,top: 22%; left: 40%;,34370,top: 80%; left: 40%;,262523,top: 61%; left: 73%;,88682,...,top: 36%; left: 40%;,40428,top: 33%; left: 25%;,3-5-2 flat,2127,410,2,2-3,2,3
1,948786,113974,top: 80%; left: 40%;,113979,top: 61%; left: 7.5%;,59373,top: 61%; left: 73%;,57628,top: 63%; left: 28%;,68152,...,top: 3%; left: 40%;,38267,top: 63%; left: 28%;,4-2-3-1,97,3817,0,1-1,1,1
2,1134605,88983,top: 2%; left: 30%;,143219,top: 2%; left: 50%;,45601,top: 80%; left: 40%;,145922,top: 61%; left: 73%;,128912,...,top: 63%; left: 52.5%;,42413,top: 43%; left: 28%;,4-2-3-1,3866,9251,1,4-0,4,0
3,2463048,58030,top: 18%; left: 65%;,78147,top: 3%; left: 40%;,93763,top: 80%; left: 40%;,56112,top: 43%; left: 40%;,45677,...,top: 43%; left: 52%;,39998,top: 23%; left: 68%;,4-4-2 double 6,433,269,1,1-0,1,0
4,2547389,183120,top: 23%; left: 40%;,42393,top: 3%; left: 40%;,128969,top: 80%; left: 40%;,110668,top: 61%; left: 7.5%;,19312,...,top: 61%; left: 7.5%;,24049,top: 43%; left: 52%;,4-2-3-1,5191,339,1,1-0,1,0
5,2463041,41118,top: 63%; left: 52.5%;,18296,top: 3%; left: 40%;,89649,top: 80%; left: 40%;,31608,top: 63%; left: 28%;,83415,...,top: 23%; left: 40%;,84425,top: 18%; left: 15%;,4-2-3-1,6092,17,0,1-1,1,1
6,1143249,6053,top: 2%; left: 50%;,33570,top: 2%; left: 30%;,27539,top: 80%; left: 40%;,58816,top: 61%; left: 73%;,58821,...,top: 25%; left: 12%;,7153,top: 35%; left: 25%;,4-5-1,789,2381,2,0-4,0,4
7,1143248,16499,top: 18%; left: 65%;,174984,top: 3%; left: 40%;,14555,top: 80%; left: 40%;,44996,top: 63%; left: 28%;,16353,...,top: 43%; left: 52%;,5628,top: 10%; left: 15%;,4-3-3 def.,806,3216,2,1-3,1,3
8,1143245,37091,top: 2%; left: 30%;,38768,top: 2%; left: 50%;,28124,top: 80%; left: 40%;,4873,top: 63%; left: 28%;,7402,...,top: 28%; left: 53%;,56372,top: 10%; left: 65%;,4-3-3 off.,2043,820,1,2-1,2,1
9,1143247,36873,top: 18%; left: 15%;,33833,top: 3%; left: 40%;,7110,top: 80%; left: 40%;,16698,top: 61%; left: 73%;,27280,...,top: 43%; left: 52%;,8539,top: 43%; left: 28%;,4-4-2 double 6,5199,2323,1,1-0,1,0


In [14]:
nemkap1=concede[concede['score0']=='0']
nemkap2=concede[concede['score1']=='0']

KeyError: 'score0'

In [ ]:
df1=nemkap2[['away10_id','away11_id','away1_id','away2_id','away3_id','away4_id','away5_id','away6_id','away7_id','away8_id','away9_id']].unstack().value_counts()
df1

In [ ]:
df2=nemkap1[['home10_id','home11_id','home1_id','home2_id','home3_id','home4_id','home5_id','home6_id','home7_id','home8_id','home9_id']].unstack().value_counts()
df2

In [ ]:
#holtverseny van az első kettő között
print('playerid:',df1.add(df2).sort_values(ascending=False).index[0],'meccsek száma:',df1.add(df2).sort_values(ascending=False).iloc[0])
print('playerid:',df1.add(df2).sort_values(ascending=False).index[1],'meccsek száma:',df1.add(df2).sort_values(ascending=False).iloc[1])

# position-with-largest-average-height

In [15]:
haho=[]
for i in range(1,12):
        haho.append(('df%s' %i))
        #df_height[['home%s_id' %i,'home%s_pos' %i]]
        #('df%s' %i).columns=['playerid','pos']
(haho)

['df1', 'df2', 'df3', 'df4', 'df5', 'df6', 'df7', 'df8', 'df9', 'df10', 'df11']

In [16]:
df_height=m_i_df[a]
frames=[]
def tablagenerator():
    for i in range(1,12):
        haho[i-1]=df_height[['home%s_id' %i,'home%s_pos' %i]]
        haho[i-1].columns=['playerid','pos']
        frames.append(haho[i-1])
        result = pd.concat(frames,axis=0,ignore_index=False)
    return result
df_playerid=tablagenerator()

# Pozició amiben átlagosan a legmagasabbak a játékosok

In [17]:
df_height

,home1_id,home1_pos,away1_id,away1_pos,home2_id,home2_pos,away2_id,away2_pos,home3_id,home3_pos,...,away9_id,away9_pos,home10_id,home10_pos,away10_id,away10_pos,home11_id,home11_pos,away11_id,away11_pos
0,16408,top: 80%; left: 40%;,34370,top: 80%; left: 40%;,261611,top: 27%; left: 10%;,262523,top: 61%; left: 73%;,6750,top: 62%; left: 40%;,...,119235,top: 2%; left: 50%;,6031,top: 2%; left: 50%;,45146,top: 2%; left: 30%;,29316,top: 2%; left: 30%;,66521,top: 22%; left: 40%;
1,3612,top: 80%; left: 40%;,59373,top: 61%; left: 73%;,19354,top: 61%; left: 7.5%;,57628,top: 63%; left: 28%;,26485,top: 63%; left: 52.5%;,...,59352,top: 63%; left: 52.5%;,16474,top: 23%; left: 40%;,113974,top: 80%; left: 40%;,31173,top: 43%; left: 28%;,113979,top: 61%; left: 7.5%;
2,73472,top: 80%; left: 40%;,45601,top: 80%; left: 40%;,141655,top: 61%; left: 7.5%;,145922,top: 61%; left: 73%;,118534,top: 63%; left: 28%;,...,103819,top: 63%; left: 28%;,62049,top: 3%; left: 40%;,88983,top: 2%; left: 30%;,61887,top: 18%; left: 15%;,143219,top: 2%; left: 50%;
3,18971,top: 80%; left: 40%;,93763,top: 80%; left: 40%;,57966,top: 63%; left: 28%;,56112,top: 43%; left: 40%;,11583,top: 63%; left: 52.5%;,...,46469,top: 23%; left: 50%;,31968,top: 23%; left: 12%;,58030,top: 18%; left: 65%;,48547,top: 2%; left: 30%;,78147,top: 3%; left: 40%;
4,91931,top: 80%; left: 40%;,128969,top: 80%; left: 40%;,57987,top: 63%; left: 28%;,110668,top: 61%; left: 7.5%;,101304,top: 63%; left: 52.5%;,...,24484,top: 18%; left: 65%;,97567,top: 23%; left: 40%;,183120,top: 23%; left: 40%;,59908,top: 3%; left: 40%;,42393,top: 3%; left: 40%;
5,104220,top: 80%; left: 40%;,89649,top: 80%; left: 40%;,45685,top: 63%; left: 28%;,31608,top: 63%; left: 28%;,134636,top: 61%; left: 7.5%;,...,86895,top: 23%; left: 40%;,5036,top: 43%; left: 28%;,41118,top: 63%; left: 52.5%;,164265,top: 3%; left: 40%;,18296,top: 3%; left: 40%;
6,67797,top: 80%; left: 40%;,27539,top: 80%; left: 40%;,85302,top: 63%; left: 28%;,58816,top: 61%; left: 73%;,29678,top: 61%; left: 73%;,...,99186,top: 23%; left: 12%;,10364,top: 30%; left: 40%;,6053,top: 2%; left: 50%;,49838,top: 3%; left: 40%;,33570,top: 2%; left: 30%;
7,16693,top: 80%; left: 40%;,14555,top: 80%; left: 40%;,138409,top: 63%; left: 28%;,44996,top: 63%; left: 28%;,33794,top: 61%; left: 73%;,...,77737,top: 43%; left: 28%;,15821,top: 3%; left: 40%;,16499,top: 18%; left: 65%;,66405,top: 10%; left: 65%;,174984,top: 3%; left: 40%;
8,85339,top: 80%; left: 40%;,28124,top: 80%; left: 40%;,37722,top: 63%; left: 28%;,4873,top: 63%; left: 28%;,7005,top: 63%; left: 52.5%;,...,29627,top: 23%; left: 68%;,31867,top: 10%; left: 15%;,37091,top: 2%; left: 30%;,51350,top: 3%; left: 40%;,38768,top: 2%; left: 50%;
9,22109,top: 80%; left: 40%;,7110,top: 80%; left: 40%;,7152,top: 63%; left: 28%;,16698,top: 61%; left: 73%;,138476,top: 63%; left: 52.5%;,...,90937,top: 43%; left: 40%;,46658,top: 2%; left: 30%;,36873,top: 18%; left: 15%;,6924,top: 2%; left: 50%;,33833,top: 3%; left: 40%;


In [18]:
df_playerid

,playerid,pos
0,16408,top: 80%; left: 40%;
1,3612,top: 80%; left: 40%;
2,73472,top: 80%; left: 40%;
3,18971,top: 80%; left: 40%;
4,91931,top: 80%; left: 40%;
5,104220,top: 80%; left: 40%;
6,67797,top: 80%; left: 40%;
7,16693,top: 80%; left: 40%;
8,85339,top: 80%; left: 40%;
9,22109,top: 80%; left: 40%;


In [19]:
#tesztkor=pd.DataFrame(p_i_df['height'].apply(stringtoint).dropna())
average_height=pd.merge(df_playerid,p_i_df,left_on='playerid',right_on='playerid')#.groupby('pos').agg({'height':['mean','count']})
average_height=average_height.dropna().drop_duplicates()
average_height['height']=average_height['height'].apply(stringtoint)
#average_height
average_height.groupby('pos').agg({'height':'mean'}).sort_values('height',ascending=False).index[0]
#average_height

'top: 80%; left: 40%;'

In [20]:
average_height

,playerid,pos,dob,height
0,16408,top: 80%; left: 40%;,1985-07-11,190.0
98,3612,top: 80%; left: 40%;,1986-07-02,187.0
150,73472,top: 80%; left: 40%;,1990-04-10,194.0
216,18971,top: 80%; left: 40%;,1985-10-14,189.0
267,91931,top: 80%; left: 40%;,1988-01-29,194.0
316,104220,top: 80%; left: 40%;,1994-01-22,192.0
346,67797,top: 80%; left: 40%;,1987-10-23,195.0
408,16693,top: 80%; left: 40%;,1982-08-17,196.0
465,85339,top: 80%; left: 40%;,1991-01-16,190.0
487,22109,top: 80%; left: 40%;,1982-12-11,192.0


# Position with youngest average age

In [21]:
average_height=average_height.assign(y_as_d = lambda df: pd.to_datetime(average_height['dob'], format = '%Y-%m-%d'))

In [22]:
youngest_age=average_height.drop_duplicates()
youngest_age

,playerid,pos,dob,height,y_as_d
0,16408,top: 80%; left: 40%;,1985-07-11,190.0,1985-07-11
98,3612,top: 80%; left: 40%;,1986-07-02,187.0,1986-07-02
150,73472,top: 80%; left: 40%;,1990-04-10,194.0,1990-04-10
216,18971,top: 80%; left: 40%;,1985-10-14,189.0,1985-10-14
267,91931,top: 80%; left: 40%;,1988-01-29,194.0,1988-01-29
316,104220,top: 80%; left: 40%;,1994-01-22,192.0,1994-01-22
346,67797,top: 80%; left: 40%;,1987-10-23,195.0,1987-10-23
408,16693,top: 80%; left: 40%;,1982-08-17,196.0,1982-08-17
465,85339,top: 80%; left: 40%;,1991-01-16,190.0,1991-01-16
487,22109,top: 80%; left: 40%;,1982-12-11,192.0,1982-12-11


In [23]:
youngest_age=youngest_age.assign(
    fiatal = lambda youngest_age: pd.Timestamp.now()-youngest_age['y_as_d']).sort_values('fiatal')

In [24]:
pd.Timestamp.now()

Timestamp('2019-04-14 22:36:23.735855')

In [25]:
youngest_age['fiatal']=(youngest_age['fiatal'].dt.days/365)

In [26]:
youngest_age.groupby('pos').agg({'fiatal':'mean'}).sort_values('fiatal').index[0]

'top: 18%; left: 15%;'

# position with highest average value

In [27]:
highest_value=pd.merge(average_height, p_v_df)
highest_value

,playerid,pos,dob,height,y_as_d,date,y
0,16408,top: 80%; left: 40%;,1985-07-11,190.0,1985-07-11,2004-10-04,50000
1,16408,top: 80%; left: 40%;,1985-07-11,190.0,1985-07-11,2005-11-06,100000
2,16408,top: 80%; left: 40%;,1985-07-11,190.0,1985-07-11,2007-02-05,150000
3,16408,top: 80%; left: 40%;,1985-07-11,190.0,1985-07-11,2007-08-27,200000
4,16408,top: 80%; left: 40%;,1985-07-11,190.0,1985-07-11,2007-09-03,250000
5,16408,top: 80%; left: 40%;,1985-07-11,190.0,1985-07-11,2011-02-28,250000
6,16408,top: 80%; left: 40%;,1985-07-11,190.0,1985-07-11,2011-07-02,250000
7,16408,top: 80%; left: 40%;,1985-07-11,190.0,1985-07-11,2011-11-21,250000
8,16408,top: 80%; left: 40%;,1985-07-11,190.0,1985-07-11,2012-02-26,1000000
9,16408,top: 80%; left: 40%;,1985-07-11,190.0,1985-07-11,2012-09-21,1500000


In [28]:
highest_value.groupby('pos').agg({'y':'mean'}).sort_values('y',ascending=False)

,y
pos,
top: 20%; left: 30%;,2.829177e+06
top: 20%; left: 50%;,2.705045e+06
top: 22%; left: 40%;,2.366156e+06
top: 29%; left: 26%;,2.213537e+06
top: 28%; left: 27%;,2.135502e+06
top: 10%; left: 15%;,2.113666e+06
top: 30%; left: 40%;,2.112280e+06
top: 42%; left: 15%;,2.102294e+06
top: 10%; left: 65%;,2.085686e+06


# longest-time-in-days-between-two-games-for-player

In [29]:
idk=[]
idk.append('date')
for i in range(1,12):
    idk.append('home%s_id' %i)
    idk.append('away%s_id' %i)
idk

['date',
 'home1_id',
 'away1_id',
 'home2_id',
 'away2_id',
 'home3_id',
 'away3_id',
 'home4_id',
 'away4_id',
 'home5_id',
 'away5_id',
 'home6_id',
 'away6_id',
 'home7_id',
 'away7_id',
 'home8_id',
 'away8_id',
 'home9_id',
 'away9_id',
 'home10_id',
 'away10_id',
 'home11_id',
 'away11_id']

In [30]:
frames2=[]
def tablagenerator2():
    for i in range(1,12):
        idk[i]=m_i_df[['home%s_id' %i,'date']]
        idk[i].columns=['playerid','date']
        frames2.append(idk[i])
    for j in range(1,12):
        idk[j]=m_i_df[['away%s_id' %j,'date']]
        idk[j].columns=['playerid','date']
        frames2.append(idk[j])
        
    result = pd.concat(frames2,axis=0,ignore_index=True)
    return result
longest_df=tablagenerator2()
longest_df

,playerid,date
0,16408,2015-09-22
1,3612,2009-03-31
2,73472,2011-12-15
3,18971,2014-11-01
4,91931,2015-04-23
5,104220,2014-10-26
6,67797,2011-09-18
7,16693,2011-09-18
8,85339,2011-09-17
9,22109,2011-09-17


In [31]:
longest_df=longest_df.assign(y_as_d = lambda df: pd.to_datetime(longest_df['date'], format = '%Y-%m-%d'))

In [32]:
longest_df=longest_df.sort_values(['playerid','y_as_d'])
longest_df

,playerid,date,y_as_d
2654194,1,2000-11-01,2000-11-01
674721,1,2000-11-09,2000-11-09
2470384,1,2001-08-24,2001-08-24
1003013,1,2001-12-02,2001-12-02
3174321,1,2001-12-07,2001-12-07
2698109,1,2002-02-09,2002-02-09
2654616,1,2002-07-27,2002-07-27
2998200,1,2002-09-14,2002-09-14
896971,1,2003-01-25,2003-01-25
2900735,1,2003-02-01,2003-02-01


In [33]:
lista = longest_df['y_as_d'].tolist()

In [34]:
pl = longest_df['playerid'].tolist()

In [35]:
kul_lista = [0]
for i in range(len(lista)):
    if i == 0:
        continue
    else:
        if pl[i] == pl[i-1]:
            kul_lista.append(lista[i]- lista[i-1])
        else:
            kul_lista.append(0)

In [86]:
kul_lista

[0,
 Timedelta('8 days 00:00:00'),
 Timedelta('288 days 00:00:00'),
 Timedelta('100 days 00:00:00'),
 Timedelta('5 days 00:00:00'),
 Timedelta('64 days 00:00:00'),
 Timedelta('168 days 00:00:00'),
 Timedelta('49 days 00:00:00'),
 Timedelta('133 days 00:00:00'),
 Timedelta('7 days 00:00:00'),
 Timedelta('71 days 00:00:00'),
 Timedelta('3 days 00:00:00'),
 Timedelta('5 days 00:00:00'),
 Timedelta('190 days 00:00:00'),
 Timedelta('285 days 00:00:00'),
 Timedelta('99 days 00:00:00'),
 0,
 Timedelta('6 days 00:00:00'),
 Timedelta('33 days 00:00:00'),
 Timedelta('6 days 00:00:00'),
 Timedelta('3 days 00:00:00'),
 Timedelta('12 days 00:00:00'),
 Timedelta('5 days 00:00:00'),
 Timedelta('37 days 00:00:00'),
 Timedelta('42 days 00:00:00'),
 Timedelta('7 days 00:00:00'),
 Timedelta('7 days 00:00:00'),
 Timedelta('8 days 00:00:00'),
 Timedelta('4 days 00:00:00'),
 Timedelta('3 days 00:00:00'),
 Timedelta('6 days 00:00:00'),
 Timedelta('5 days 00:00:00'),
 Timedelta('3 days 00:00:00'),
 Timedelta(

In [87]:
max([f for f in kul_lista if not type(f) == int])

Timedelta('5179 days 00:00:00')

# biggest-stadium-capacity-difference-upset - kisebb stadionú csapat nyer

In [88]:
m_i_df[['home_team','away_team', 'score0','score1']]

,home_team,away_team,score0,score1
0,410,5,2,3
1,3817,22247,1,1
2,9251,9252,4,0
3,269,10,1,0
4,339,2282,1,0
5,17,30,1,1
6,2381,825,0,4
7,3216,20,1,3
8,820,1506,2,1
9,2323,1267,1,0


In [89]:
biggest0=pd.merge(t_i_df,m_i_df[['home_team','away_team', 'score0','score1']],left_on='team_id',right_on='home_team')

In [90]:
biggest=pd.merge(biggest0,t_i_df,left_on='away_team', right_on='team_id')
biggest['seat_diff']=(biggest['seats_x']-biggest['seats_y'])
biggest.dropna()[biggest['seat_diff']<0].sort_values('seat_diff')[biggest['score0']>biggest['score1']]

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  This is separate from the ipykernel package so we can avoid doing imports until


,team_id_x,name_x,seats_x,home_team,away_team,score0,score1,team_id_y,name_y,seats_y,seat_diff
14490,2296,CD Numancia,9025.0,2296,131,1,0,131,FC Barcelona,99354.0,-90329.0
150637,23285,University of Pretoria FC.,8000.0,23285,568,1,0,568,Kaizer Chiefs,94700.0,-86700.0
14510,3709,Getafe CF,14400.0,3709,131,1,0,131,FC Barcelona,99354.0,-84954.0
14327,265,Panathinaikos Athens,16620.0,265,131,1,0,131,FC Barcelona,99354.0,-82734.0
150630,14187,Maritzburg United FC,12000.0,14187,568,1,0,568,Kaizer Chiefs,94700.0,-82700.0
90688,629,Olimpia Asunción,0.0,629,2462,2,1,2462,Fluminense Football Club,82238.0,-82238.0
24822,22702,Masr El Makasa,0.0,22702,9094,3,2,9094,Masry Port Said,80000.0,-80000.0
24810,9219,Tala'ea El Gaish,0.0,9219,9094,2,1,9094,Masry Port Said,80000.0,-80000.0
144008,3229,Al-Rayyan Sports Club,0.0,3229,1076,1,0,1076,Esteghlal FC,80000.0,-80000.0
150616,10680,Bidvest Wits FC,15000.0,10680,568,2,1,568,Kaizer Chiefs,94700.0,-79700.0


In [91]:
print(biggest.dropna()[biggest['seat_diff']<0].sort_values('seat_diff')[biggest['score0']>biggest['score1']].iloc[0],'\n','\n','\n', 'biggest_stadium_capacity_difference_upset',\
      abs(biggest.dropna()[biggest['seat_diff']<0].sort_values('seat_diff')[biggest['score0']>biggest['score1']]['seat_diff'].iloc[0]))

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


team_id_x            2296
name_x        CD Numancia
seats_x              9025
home_team            2296
away_team             131
score0                  1
score1                  0
team_id_y             131
name_y       FC Barcelona
seats_y             99354
seat_diff          -90329
Name: 14490, dtype: object 
 
 
 biggest_stadium_capacity_difference_upset 90329.0


# Capacity of stadium of team with most games

In [92]:
home_df=biggest['name_x'].value_counts()
home_df.head(5)

Bayern Munich          367
Bayer 04 Leverkusen    339
Galatasaray SK         315
Borussia Dortmund      313
1.FC Kaiserslautern    287
Name: name_x, dtype: int64

In [93]:
away_df=biggest['name_y'].value_counts()
away_df.head(5)

Bayern Munich          381
Bayer 04 Leverkusen    363
Borussia Dortmund      323
1.FC Kaiserslautern    309
Galatasaray SK         294
Name: name_y, dtype: int64

In [94]:
print(home_df.add(away_df).sort_values(ascending=False).index[0],home_df.add(away_df).sort_values(ascending=False)[0])
capa=home_df.add(away_df).sort_values(ascending=False).index[0]

Bayern Munich 748.0


In [95]:
print('Capacity of stadium of team with most games:',biggest[biggest['name_x']==capa]['seats_x'].iloc[0])

Capacity of stadium of team with most games: 75000.0


# biggest-value-difference -  csapatok összesített játékos-értéke közötti különbség


In [96]:
# P_V_DF + MIDF CSAPATOK ALAPJÁN MECCSENKÉNTI ÉRTÉKKÜLÖNBSÉG

In [13]:
m_i_df

,mkey,away10_id,away10_pos,away11_id,away11_pos,away1_id,away1_pos,away2_id,away2_pos,away3_id,...,home7_pos,home8_id,home8_pos,home9_id,home9_pos,home_formation,home_manager,home_team,result,score
0,2604022,45146,top: 2%; left: 30%;,66521,top: 22%; left: 40%;,34370,top: 80%; left: 40%;,262523,top: 61%; left: 73%;,88682,...,top: 33%; left: 55%;,240306,top: 36%; left: 40%;,40428,top: 33%; left: 25%;,3-5-2 flat,2127,410,2,2-3
1,948786,113974,top: 80%; left: 40%;,113979,top: 61%; left: 7.5%;,59373,top: 61%; left: 73%;,57628,top: 63%; left: 28%;,68152,...,top: 18%; left: 65%;,39743,top: 3%; left: 40%;,38267,top: 63%; left: 28%;,4-2-3-1,97,3817,0,1-1
2,1134605,88983,top: 2%; left: 30%;,143219,top: 2%; left: 50%;,45601,top: 80%; left: 40%;,145922,top: 61%; left: 73%;,128912,...,top: 43%; left: 52%;,128262,top: 63%; left: 52.5%;,42413,top: 43%; left: 28%;,4-2-3-1,3866,9251,1,4-0
3,2463048,58030,top: 18%; left: 65%;,78147,top: 3%; left: 40%;,93763,top: 80%; left: 40%;,56112,top: 43%; left: 40%;,45677,...,top: 43%; left: 28%;,85927,top: 43%; left: 52%;,39998,top: 23%; left: 68%;,4-4-2 double 6,433,269,1,1-0
4,2547389,183120,top: 23%; left: 40%;,42393,top: 3%; left: 40%;,128969,top: 80%; left: 40%;,110668,top: 61%; left: 7.5%;,19312,...,top: 18%; left: 15%;,22378,top: 61%; left: 7.5%;,24049,top: 43%; left: 52%;,4-2-3-1,5191,339,1,1-0
5,2463041,41118,top: 63%; left: 52.5%;,18296,top: 3%; left: 40%;,89649,top: 80%; left: 40%;,31608,top: 63%; left: 28%;,83415,...,top: 18%; left: 65%;,89654,top: 23%; left: 40%;,84425,top: 18%; left: 15%;,4-2-3-1,6092,17,0,1-1
6,1143249,6053,top: 2%; left: 50%;,33570,top: 2%; left: 30%;,27539,top: 80%; left: 40%;,58816,top: 61%; left: 73%;,58821,...,top: 35%; left: 55%;,53556,top: 25%; left: 12%;,7153,top: 35%; left: 25%;,4-5-1,789,2381,2,0-4
7,1143248,16499,top: 18%; left: 65%;,174984,top: 3%; left: 40%;,14555,top: 80%; left: 40%;,44996,top: 63%; left: 28%;,16353,...,top: 43%; left: 28%;,44992,top: 43%; left: 52%;,5628,top: 10%; left: 15%;,4-3-3 def.,806,3216,2,1-3
8,1143245,37091,top: 2%; left: 30%;,38768,top: 2%; left: 50%;,28124,top: 80%; left: 40%;,4873,top: 63%; left: 28%;,7402,...,top: 61%; left: 7.5%;,21494,top: 28%; left: 53%;,56372,top: 10%; left: 65%;,4-3-3 off.,2043,820,1,2-1
9,1143247,36873,top: 18%; left: 15%;,33833,top: 3%; left: 40%;,7110,top: 80%; left: 40%;,16698,top: 61%; left: 73%;,27280,...,top: 23%; left: 68%;,80696,top: 43%; left: 52%;,8539,top: 43%; left: 28%;,4-4-2 double 6,5199,2323,1,1-0


In [17]:
meccsek_df=m_i_df[['mkey','home_team','away_team','date']]
meccsek_df

,mkey,home_team,away_team,date
0,2604022,410,5,2015-09-22
1,948786,3817,22247,2009-03-31
2,1134605,9251,9252,2011-12-15
3,2463048,269,10,2014-11-01
4,2547389,339,2282,2015-04-23
5,2463041,17,30,2014-10-26
6,1143249,2381,825,2011-09-18
7,1143248,3216,20,2011-09-18
8,1143245,820,1506,2011-09-17
9,1143247,2323,1267,2011-09-17


In [7]:
frames3=[]
def tablagenerator3():
    for i in range(1,12):
        idk[i]=m_i_df[['home%s_id' %i,'home_team','mkey','date']]
        idk[i].columns=['playerid','team_id','mkey','mdate']
        frames3.append(idk[i])
    for j in range(1,12):
        idk[j]=m_i_df[['away%s_id' %j,'away_team','mkey','date']]
        idk[j].columns=['playerid','team_id','mkey','mdate']
        frames3.append(idk[j])
        
    result = pd.concat(frames3,axis=0,ignore_index=True)
    return result
team_player_id=tablagenerator3().drop_duplicates()
team_player_id

,playerid,team_id,mkey,mdate
0,16408,410,2604022,2015-09-22
1,3612,3817,948786,2009-03-31
2,73472,9251,1134605,2011-12-15
3,18971,269,2463048,2014-11-01
4,91931,339,2547389,2015-04-23
5,104220,17,2463041,2014-10-26
6,67797,2381,1143249,2011-09-18
7,16693,3216,1143248,2011-09-18
8,85339,820,1143245,2011-09-17
9,22109,2323,1143247,2011-09-17


In [8]:
team_value_df=pd.merge(team_player_id,p_v_df)
team_value_df=team_value_df.assign(y_as_d = lambda df: pd.to_datetime(team_value_df['mdate'], format = '%Y-%m-%d'))\
.drop(columns=['mdate'])
team_value_df

,playerid,team_id,mkey,date,y,y_as_d
0,16408,410,2604022,2004-10-04,50000,2015-09-22
1,16408,410,2604022,2005-11-06,100000,2015-09-22
2,16408,410,2604022,2007-02-05,150000,2015-09-22
3,16408,410,2604022,2007-08-27,200000,2015-09-22
4,16408,410,2604022,2007-09-03,250000,2015-09-22
5,16408,410,2604022,2011-02-28,250000,2015-09-22
6,16408,410,2604022,2011-07-02,250000,2015-09-22
7,16408,410,2604022,2011-11-21,250000,2015-09-22
8,16408,410,2604022,2012-02-26,1000000,2015-09-22
9,16408,410,2604022,2012-09-21,1500000,2015-09-22


In [73]:
csapatertek=team_value_df.groupby(['team_id','mkey','y_as_d']).agg({'y':'mean'}).reset_index()#.set_index('team_id')
csapatertek
pd.merge(csapatertek,meccsek_df)

,team_id,mkey,y_as_d,y,home_team,away_team,date
0,1,3166,2003-11-07,1.788194e+05,108,1,2003-11-07
1,108,3166,2003-11-07,2.938953e+05,108,1,2003-11-07
2,1,6739,2004-05-22,1.763158e+05,1,108,2004-05-22
3,108,6739,2004-05-22,2.896341e+05,1,108,2004-05-22
4,1,20089,2002-10-03,1.195312e+05,105,1,2002-10-03
5,105,20089,2002-10-03,1.205357e+05,105,1,2002-10-03
6,1,20415,2002-11-23,2.733974e+05,108,1,2002-11-23
7,108,20415,2002-11-23,7.590517e+05,108,1,2002-11-23
8,1,21039,2004-08-08,3.598739e+05,1,111,2004-08-08
9,111,21039,2004-08-08,1.886986e+05,1,111,2004-08-08


In [76]:
csapatertek

,team_id,mkey,y_as_d,y
0,1,3166,2003-11-07,178819.444444
1,1,6739,2004-05-22,176315.789474
2,1,20089,2002-10-03,119531.250000
3,1,20415,2002-11-23,273397.435897
4,1,21039,2004-08-08,359873.949580
5,1,21075,2004-09-19,384615.384615
6,1,21151,2004-11-12,337951.807229
7,1,21228,2005-02-20,425742.574257
8,1,27352,2004-08-21,345454.545455
9,1,31217,2005-08-06,451906.779661


In [31]:
biggest_val_diff=pd.merge(csapatertek,meccsek_df).groupby('mkey').agg({'y':['min','max']})#.assign(diff = {'y':'max'}-{'y':"min"})

In [64]:
biggest_val_diff.set_axis(['min','max'], axis='columns', inplace=False)

,min,max
mkey,,
1,3.069630e+06,7.481555e+06
7,1.280660e+06,6.781793e+06
8,2.421203e+06,1.030368e+07
10,2.141239e+06,7.672443e+06
45,5.182519e+06,7.406697e+06
46,6.851471e+06,8.661399e+06
49,7.380117e+06,8.179911e+06
50,3.069630e+06,7.705755e+06
53,5.427000e+06,7.126018e+06


In [70]:
biggest_val_diff['diff']=biggest_val_diff['max']-biggest_val_diff['min']
biggest_val_diff.sort_values('diff', ascending=False)['diff'].iloc[0]

30709883.814102564

# biggest-value-difference-upset

# Median of winning team average age

In [129]:
m_i_df['winning_team']=np.where(m_i_df['score0']<m_i_df['score1'], m_i_df['away_team'], \
                                np.where(m_i_df['score0']>m_i_df['score1'], m_i_df['home_team'], float('nan')))
m_i_df

,mkey,away10_id,away10_pos,away11_id,away11_pos,away1_id,away1_pos,away2_id,away2_pos,away3_id,...,home9_id,home9_pos,home_formation,home_manager,home_team,result,score,score0,score1,winning_team
0,2604022,45146,top: 2%; left: 30%;,66521,top: 22%; left: 40%;,34370,top: 80%; left: 40%;,262523,top: 61%; left: 73%;,88682,...,40428,top: 33%; left: 25%;,3-5-2 flat,2127,410,2,2-3,2,3,5.0
1,948786,113974,top: 80%; left: 40%;,113979,top: 61%; left: 7.5%;,59373,top: 61%; left: 73%;,57628,top: 63%; left: 28%;,68152,...,38267,top: 63%; left: 28%;,4-2-3-1,97,3817,0,1-1,1,1,NaN
2,1134605,88983,top: 2%; left: 30%;,143219,top: 2%; left: 50%;,45601,top: 80%; left: 40%;,145922,top: 61%; left: 73%;,128912,...,42413,top: 43%; left: 28%;,4-2-3-1,3866,9251,1,4-0,4,0,9251.0
3,2463048,58030,top: 18%; left: 65%;,78147,top: 3%; left: 40%;,93763,top: 80%; left: 40%;,56112,top: 43%; left: 40%;,45677,...,39998,top: 23%; left: 68%;,4-4-2 double 6,433,269,1,1-0,1,0,269.0
4,2547389,183120,top: 23%; left: 40%;,42393,top: 3%; left: 40%;,128969,top: 80%; left: 40%;,110668,top: 61%; left: 7.5%;,19312,...,24049,top: 43%; left: 52%;,4-2-3-1,5191,339,1,1-0,1,0,339.0
5,2463041,41118,top: 63%; left: 52.5%;,18296,top: 3%; left: 40%;,89649,top: 80%; left: 40%;,31608,top: 63%; left: 28%;,83415,...,84425,top: 18%; left: 15%;,4-2-3-1,6092,17,0,1-1,1,1,NaN
6,1143249,6053,top: 2%; left: 50%;,33570,top: 2%; left: 30%;,27539,top: 80%; left: 40%;,58816,top: 61%; left: 73%;,58821,...,7153,top: 35%; left: 25%;,4-5-1,789,2381,2,0-4,0,4,825.0
7,1143248,16499,top: 18%; left: 65%;,174984,top: 3%; left: 40%;,14555,top: 80%; left: 40%;,44996,top: 63%; left: 28%;,16353,...,5628,top: 10%; left: 15%;,4-3-3 def.,806,3216,2,1-3,1,3,20.0
8,1143245,37091,top: 2%; left: 30%;,38768,top: 2%; left: 50%;,28124,top: 80%; left: 40%;,4873,top: 63%; left: 28%;,7402,...,56372,top: 10%; left: 65%;,4-3-3 off.,2043,820,1,2-1,2,1,820.0
9,1143247,36873,top: 18%; left: 15%;,33833,top: 3%; left: 40%;,7110,top: 80%; left: 40%;,16698,top: 61%; left: 73%;,27280,...,8539,top: 43%; left: 28%;,4-4-2 double 6,5199,2323,1,1-0,1,0,2323.0


In [130]:
frames4=[]
def tablagenerator4():
    for i in range(1,12):
        idk[i]=m_i_df[['home%s_id' %i,'home_team','date','winning_team']]
        idk[i].columns=['playerid','team_id','mdate','winning_team']
        frames4.append(idk[i])
    for j in range(1,12):
        idk[j]=m_i_df[['away%s_id' %j,'away_team','date','winning_team']]
        idk[j].columns=['playerid','team_id','mdate','winning_team']
        frames4.append(idk[j])
        
    result = pd.concat(frames4,axis=0,ignore_index=True)
    return result
avg_age=tablagenerator4().drop_duplicates()

In [131]:
avg_age=avg_age.assign(y_as_d = lambda df: pd.to_datetime(avg_age['mdate'], format = '%Y-%m-%d'))\
.drop(columns=['mdate'])
avg_age

,playerid,team_id,winning_team,y_as_d
0,16408,410,5.0,2015-09-22
1,3612,3817,NaN,2009-03-31
2,73472,9251,9251.0,2011-12-15
3,18971,269,269.0,2014-11-01
4,91931,339,339.0,2015-04-23
5,104220,17,NaN,2014-10-26
6,67797,2381,825.0,2011-09-18
7,16693,3216,20.0,2011-09-18
8,85339,820,820.0,2011-09-17
9,22109,2323,2323.0,2011-09-17


In [132]:
seged=p_i_df[['dob','playerid']].assign(player_date = lambda df: pd.to_datetime(p_i_df['dob'], format= '%Y-%m-%d')).drop(columns=['dob'])

In [135]:
avg_age=pd.merge(avg_age,seged)

avg_age=avg_age.assign(eletkor=lambda df: avg_age['y_as_d']-avg_age['player_date'])
avg_age['eletkor'] = avg_age['eletkor'].dt.days
avg_age

,playerid,team_id,winning_team,y_as_d,player_date,eletkor
0,16408,410,5.0,2015-09-22,1985-07-11,11030.0
1,16408,410,1038.0,2015-05-10,1985-07-11,10895.0
2,16408,410,NaN,2016-10-31,1985-07-11,11435.0
3,16408,410,410.0,2014-09-29,1985-07-11,10672.0
4,16408,410,NaN,2014-11-23,1985-07-11,10727.0
5,16408,410,NaN,2015-02-01,1985-07-11,10797.0
6,16408,265,265.0,2012-01-22,1985-07-11,9691.0
7,16408,410,46.0,2015-04-28,1985-07-11,10883.0
8,16408,410,NaN,2015-11-01,1985-07-11,11070.0
9,16408,410,276.0,2014-12-14,1985-07-11,10748.0


In [136]:
avg_age=avg_age.dropna()
avg_age['iswinner']=np.where(avg_age['team_id']==avg_age['winning_team'], True, False)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [165]:
print('median-of-winning-team-avg-age:',avg_age[avg_age['iswinner']==True].groupby(['team_id','y_as_d']).agg({'eletkor':'mean'})\
.reset_index()['eletkor'].median()/365)

median-of-winning-team-avg-age: 26.265753424657536


# a legtöbb kapott gól nélküli meccset lehozó kapus

In [204]:
frames=[]
def tablagenerator5():
    for i in range(1,12):
        haho[i-1]=m_i_df[['home%s_id' %i,'home%s_pos' %i,'score1','date']]
        haho[i-1].columns=['playerid','pos','kapott_gol','date']
        frames.append(haho[i-1])
    for i in range(1,12):
        haho[i-1]=m_i_df[['away%s_id' %i,'away%s_pos' %i,'score0','date']]
        haho[i-1].columns=['playerid','pos','kapott_gol','date']
        frames.append(haho[i-1])
        result = pd.concat(frames,axis=0,ignore_index=False)
    return result
goalkeeper=tablagenerator5()
goalkeeper

,playerid,pos,kapott_gol,date
0,16408,top: 80%; left: 40%;,3,2015-09-22
1,3612,top: 80%; left: 40%;,1,2009-03-31
2,73472,top: 80%; left: 40%;,0,2011-12-15
3,18971,top: 80%; left: 40%;,0,2014-11-01
4,91931,top: 80%; left: 40%;,0,2015-04-23
5,104220,top: 80%; left: 40%;,1,2014-10-26
6,67797,top: 80%; left: 40%;,4,2011-09-18
7,16693,top: 80%; left: 40%;,3,2011-09-18
8,85339,top: 80%; left: 40%;,1,2011-09-17
9,22109,top: 80%; left: 40%;,0,2011-09-17


In [257]:
goalkeeper=goalkeeper[goalkeeper['pos']=='top: 80%; left: 40%;']
kapott_nulla=goalkeeper.sort_values(['playerid','date'], ascending=True)
kapott_nulla=kapott_nulla.reset_index()


In [270]:
def nulla():
    vissza=0
    out=[]
    for i, r in kapott_nulla.iterrows():
        
        if i==0:
            continue
        else:
            if kapott_nulla.loc[i,'playerid']==kapott_nulla.loc[i-1,'playerid']:
                
                if kapott_nulla.loc[i-1,'kapott_gol']=='0':
                    jelen+=1
                    if jelen>vissza:
                        vissza=jelen
                else:
                    jelen=0
            
    return print('a legtöbb kapott gól nélküli meccset hozó kapus' , vissza)
nulla()

a legtöbb kapott gól nélküli meccset hozó kapus 17


# Átlagosan legtöbb gólt kapó kapus születési dátuma és magassága

In [284]:
kapott_nulla['kapott_gol']=kapott_nulla['kapott_gol'].apply(stringtoint)

legtöbb_kapott=kapott_nulla.groupby('playerid').agg({'kapott_gol':'mean'}).sort_values('kapott_gol', ascending=False)

In [296]:
print('születési dátuma:' , legtöbb_kapott.reset_index().merge(p_i_df).loc[0,'dob'] ,'magassága: ', \
      legtöbb_kapott.reset_index().merge(p_i_df).loc[0,'height'], 'kapott gólok száma: ', legtöbb_kapott.reset_index().merge(p_i_df).loc[0,'kapott_gol'])

születési dátuma: 1973-04-24 magassága:  194 kapott gólok száma:  17.0


In [ ]:
list(map(lambda x:x**2, range(10)))

In [356]:
[x**2 for x in range(10) if x%2==0]

[0, 4, 16, 36, 64]

In [357]:
[(x, y) for x in [1,2,3] for y in [3,1,4] if x != y]

[(1, 3), (1, 4), (2, 3), (2, 1), (2, 4), (3, 1), (3, 4)]